# EDA Notebook — All 12 Datasets
This notebook is an **end-to-end EDA template** for 12 CSV datasets commonly used in ICU/EHR analysis (MIMIC-style). It is mentor-ready and includes:

- Setup & environment notes
- Per-dataset EDA: load, shape, head/tail, missingness, descriptive stats, categorical counts, timestamp detection
- Required fields + reasoning (why include / exclude)
- Visualization placeholders: correlation heatmap, distributions, time-series, barplots
- Tasks checklist for next steps (resampling, filtering stays, imputation strategy)

**How to use**: place your CSV files under `Data/Raw_data/` and run the cells. Replace file names if different.


In [32]:
# Setup dataset root to actual location
from pathlib import Path
DATA_ROOT = Path('../Data/Raw_data')
print('DATA_ROOT set to:', DATA_ROOT)

DATA_ROOT set to: ..\Data\Raw_data


## NOTE ON FILES
The notebook assumes the following 12 filenames (common in MIMIC-style data). If you have slightly different names, rename your files or update the `datasets` list in the first code cell.

- patients.csv
- admissions.csv
- icustays.csv
- chartevents.csv
- labevents.csv
- d_labitems.csv
- d_items.csv
- omr.csv
- prescriptions.csv
- outputevents.csv
- emar.csv
- microbiologyevents.csv

If you don't have a file, the per-dataset cell will skip gracefully and note the file is missing.


---

# Dataset: `patients.csv`

**Goal:** Perform a clean EDA for `patients.csv` following best-practice steps: setup & load, shape & dtypes, head/tail, missingness, descriptive stats, categorical unique counts, timestamp identification, and a short 'Required fields' analysis.


In [ ]:
# ---- EDA for patients.csv
file_path = DATA_ROOT / 'patients.csv'
if not file_path.exists():
    print('File not found:', file_path)
else:
    print('Processing file:', file_path)
    df = pd.read_csv(file_path, low_memory=False)
    print('\nShape:', df.shape)
    display(df.head())
    display(df.tail())
    # dtypes
    dtypes = df.dtypes.astype(str).rename('dtype')
    display(dtypes.to_frame())
    # missingness
    missing_counts = df.isna().sum().sort_values(ascending=False)
    missing_pct = (df.isna().mean()*100).sort_values(ascending=False)
    missing_summary = pd.DataFrame({'missing_count': missing_counts, 'missing_pct': missing_pct.round(2)})
    display(missing_summary)
    # descriptive stats
    try:
        display(df.describe(include='number').T)
    except Exception as e:
        print('Numeric describe error:', e)
    try:
        display(df.describe(include='object').T)
    except Exception as e:
        print('Object describe error:', e)
    # categorical unique counts
    cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
    if cat_cols:
        unique_summary = pd.DataFrame({c: [df[c].nunique()] for c in cat_cols}).T
        unique_summary.columns = ['n_unique']
        display(unique_summary)
    else:
        print('No categorical cols detected.')
    # timestamp detection (automatic heuristics + candidate names)
    import re
    parsed = {}
    for c in df.columns:
        try:
            _ = pd.to_datetime(df[c], errors='raise')
            parsed[c] = True
        except Exception:
            parsed[c] = False
    timestamp_cols = [c for c, ok in parsed.items() if ok]
    likely_time = re.compile(r'(time|date|charttime|storetime|admittime|dischtime|intime|outtime|edregtime|edouttime|deathtime)$', re.IGNORECASE)
    candidate_ts = [c for c in df.columns if likely_time.search(c)]
    print('\nAuto-parsed timestamp columns:', timestamp_cols)
    print('Candidate timestamp column names by pattern:', candidate_ts)
    for c in list(dict.fromkeys(timestamp_cols + candidate_ts)):
        s = pd.to_datetime(df[c], errors='coerce')
        print(c, '| min:', s.min(), '| max:', s.max(), '| #na:', s.isna().sum())
    # Quick correlations (if numeric columns < 100)
    numcols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numcols) > 1 and len(numcols) <= 100:
        corr = df[numcols].corr()
        print('\nNumeric columns correlation matrix (top-left 6x6 preview):')
        display(corr.iloc[:6, :6])
    else:
        print('\nSkipping correlation matrix (too many or too few numeric columns).')
    # Save a small sample CSV for quick inspection (first 100 rows)
    out_sample = DATA_ROOT.parent / 'Work' / 'samples'
    out_sample.mkdir(parents=True, exist_ok=True)
    sample_file = out_sample / 'patients_sample_100.csv'
    df.head(100).to_csv(sample_file, index=False)
    print('Saved sample to:', sample_file)


Timestamp-like columns: ['admittime', 'dischtime', 'deathtime', 'edregtime', 'edouttime']


### Required Fields Analysis for `patients.csv`
Paste the output of the cell above and then fill the table below by hand or run automated reasoning prompts.

```
Required fields (suggested):
- Identifiers: subject_id, hadm_id, stay_id (if present)
- Timestamps: admittime, dischtime, intime, outtime, charttime, storetime, edregtime, edouttime
- Outcome markers: deathtime, hospital_expire_flag
- Clinical: for chartevents/labevents — itemid, value, valueuom, valuenum, charttime/storetime
- Metadata: admission_type, admission_location, discharge_location, insurance, ethnicity, language
```

**Why these fields?**
- Identifiers enable joins across datasets.
- Timestamps enable LOS and time-series alignment.
- Outcome markers are targets for modeling.
- Clinical numeric fields are the actual features used for vitals and labs.

**Why other fields are lower priority?**
- Billing or administrative fields often add noise and are not useful for physiological modeling.
- High-cardinality free-text fields require dedicated NLP preprocessing before use.


---

# Dataset: `admissions.csv`

**Goal:** Perform a clean EDA for `admissions.csv` following best-practice steps: setup & load, shape & dtypes, head/tail, missingness, descriptive stats, categorical unique counts, timestamp identification, and a short 'Required fields' analysis.


In [50]:
# ---- EDA for admissions.csv
file_path = DATA_ROOT / 'admissions.csv'
if not file_path.exists():
    print('File not found:', file_path)
else:
    print('Processing file:', file_path)
    df = pd.read_csv(file_path, low_memory=False)
    print('\nShape:', df.shape)
    display(df.head())
    display(df.tail())
    # dtypes
    dtypes = df.dtypes.astype(str).rename('dtype')
    display(dtypes.to_frame())
    # missingness
    missing_counts = df.isna().sum().sort_values(ascending=False)
    missing_pct = (df.isna().mean()*100).sort_values(ascending=False)
    missing_summary = pd.DataFrame({'missing_count': missing_counts, 'missing_pct': missing_pct.round(2)})
    display(missing_summary)
    # descriptive stats
    try:
        display(df.describe(include='number').T)
    except Exception as e:
        print('Numeric describe error:', e)
    try:
        display(df.describe(include='object').T)
    except Exception as e:
        print('Object describe error:', e)
    # categorical unique counts
    cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
    if cat_cols:
        unique_summary = pd.DataFrame({c: [df[c].nunique()] for c in cat_cols}).T
        unique_summary.columns = ['n_unique']
        display(unique_summary)
    else:
        print('No categorical cols detected.')
    # timestamp detection (automatic heuristics + candidate names)
    import re
    parsed = {}
    for c in df.columns:
        try:
            _ = pd.to_datetime(df[c], errors='raise')
            parsed[c] = True
        except Exception:
            parsed[c] = False
    timestamp_cols = [c for c, ok in parsed.items() if ok]
    likely_time = re.compile(r'(time|date|charttime|storetime|admittime|dischtime|intime|outtime|edregtime|edouttime|deathtime)$', re.IGNORECASE)
    candidate_ts = [c for c in df.columns if likely_time.search(c)]
    print('\nAuto-parsed timestamp columns:', timestamp_cols)
    print('Candidate timestamp column names by pattern:', candidate_ts)
    for c in list(dict.fromkeys(timestamp_cols + candidate_ts)):
        s = pd.to_datetime(df[c], errors='coerce')
        print(c, '| min:', s.min(), '| max:', s.max(), '| #na:', s.isna().sum())
    # Quick correlations (if numeric columns < 100)
    numcols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numcols) > 1 and len(numcols) <= 100:
        corr = df[numcols].corr()
        print('\nNumeric columns correlation matrix (top-left 6x6 preview):')
        display(corr.iloc[:6, :6])
    else:
        print('\nSkipping correlation matrix (too many or too few numeric columns).')
    # Save a small sample CSV for quick inspection (first 100 rows)
    out_sample = DATA_ROOT.parent / 'Work' / 'samples'
    out_sample.mkdir(parents=True, exist_ok=True)
    sample_file = out_sample / 'admissions_sample_100.csv'
    df.head(100).to_csv(sample_file, index=False)
    print('Saved sample to:', sample_file)


Processing file: ..\Data\Raw_data\admissions.csv

Shape: (275, 16)

Shape: (275, 16)


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10004235,24181354,2196-02-24 14:38:00,2196-03-04 14:02:00,NaN,URGENT,P03YMR,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicaid,ENGLISH,SINGLE,BLACK/CAPE VERDEAN,2196-02-24 12:15:00,2196-02-24 17:07:00,0
1,10009628,25926192,2153-09-17 17:08:00,2153-09-25 13:20:00,NaN,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,?,MARRIED,HISPANIC/LATINO - PUERTO RICAN,NaN,NaN,0
2,10018081,23983182,2134-08-18 02:02:00,2134-08-23 19:35:00,NaN,URGENT,P233F6,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,2134-08-17 16:24:00,2134-08-18 03:15:00,0
3,10006053,22942076,2111-11-13 23:39:00,2111-11-15 17:20:00,2111-11-15 17:20:00,URGENT,P38TI6,TRANSFER FROM HOSPITAL,DIED,Medicaid,ENGLISH,NaN,UNKNOWN,NaN,NaN,1
4,10031404,21606243,2113-08-04 18:46:00,2113-08-06 20:57:00,NaN,URGENT,P07HDB,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,NaN,NaN,0


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
270,10038992,24745425,2187-07-29 01:05:00,2187-08-03 17:02:00,NaN,SURGICAL SAME DAY ADMISSION,P41R5N,PHYSICIAN REFERRAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,0
271,10008287,22168393,2145-09-28 01:17:00,2145-10-02 13:35:00,NaN,SURGICAL SAME DAY ADMISSION,P898NM,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,NaN,NaN,0
272,10022880,27708593,2177-03-12 07:15:00,2177-03-19 14:25:00,NaN,SURGICAL SAME DAY ADMISSION,P99698,PHYSICIAN REFERRAL,HOME,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,0
273,10004457,23251352,2141-12-17 11:00:00,2141-12-21 15:56:00,NaN,SURGICAL SAME DAY ADMISSION,P41R5N,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,SINGLE,OTHER,NaN,NaN,0
274,10004457,28108313,2147-12-19 00:00:00,2147-12-21 16:10:00,NaN,SURGICAL SAME DAY ADMISSION,P10WWR,PHYSICIAN REFERRAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,DIVORCED,WHITE,NaN,NaN,0


,dtype
subject_id,int64
hadm_id,int64
admittime,object
dischtime,object
deathtime,object
admission_type,object
admit_provider_id,object
admission_location,object
discharge_location,object
insurance,object


,missing_count,missing_pct
deathtime,260,94.55
edouttime,93,33.82
edregtime,93,33.82
discharge_location,42,15.27
marital_status,12,4.36
subject_id,0,0.00
admittime,0,0.00
hadm_id,0,0.00
admission_location,0,0.00
admit_provider_id,0,0.00


...

,n_unique
admittime,275
dischtime,275
deathtime,15
admission_type,9
admit_provider_id,170
admission_location,10
discharge_location,10
insurance,3
language,2
marital_status,4


C:\Users\Adhis\AppData\Local\Temp\ipykernel_13632\1444626279.py:41: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
C:\Users\Adhis\AppData\Local\Temp\ipykernel_13632\1444626279.py:41: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
C:\Users\Adhis\AppData\Local\Temp\ipykernel_13632\1444626279.py:41: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of i


Auto-parsed timestamp columns: ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'edregtime', 'edouttime', 'hospital_expire_flag']
Candidate timestamp column names by pattern: ['admittime', 'dischtime', 'deathtime', 'edregtime', 'edouttime']
subject_id | min: 1970-01-01 00:00:00.010000032 | max: 1970-01-01 00:00:00.010040025 | #na: 0
hadm_id | min: 1970-01-01 00:00:00.020044587 | max: 1970-01-01 00:00:00.029974575 | #na: 0
admittime | min: 2110-04-11 15:08:00 | max: 2201-12-11 12:00:00 | #na: 0
dischtime | min: 2110-04-14 15:00:00 | max: 2201-12-17 13:45:00 | #na: 0
deathtime | min: 2111-11-15 17:20:00 | max: 2201-07-13 23:27:00 | #na: 260
edregtime | min: 2110-11-30 04:45:00 | max: 2201-10-30 10:48:00 | #na: 93
edouttime | min: 2110-11-30 08:03:00 | max: 2201-10-30 12:25:00 | #na: 93
hospital_expire_flag | min: 1970-01-01 00:00:00 | max: 1970-01-01 00:00:00.000000001 | #na: 0

Numeric columns correlation matrix (top-left 6x6 preview):

Numeric columns correlation matri

,subject_id,hadm_id,hospital_expire_flag
subject_id,1.000000,0.017543,0.016616
hadm_id,0.017543,1.000000,-0.038654
hospital_expire_flag,0.016616,-0.038654,1.000000


Saved sample to: ..\Data\Work\samples\admissions_sample_100.csv


### Required Fields Analysis for `admissions.csv`
Paste the output of the cell above and then fill the table below by hand or run automated reasoning prompts.

```
Required fields (suggested):
- Identifiers: subject_id, hadm_id, stay_id (if present)
- Timestamps: admittime, dischtime, intime, outtime, charttime, storetime, edregtime, edouttime
- Outcome markers: deathtime, hospital_expire_flag
- Clinical: for chartevents/labevents — itemid, value, valueuom, valuenum, charttime/storetime
- Metadata: admission_type, admission_location, discharge_location, insurance, ethnicity, language
```

**Why these fields?**
- Identifiers enable joins across datasets.
- Timestamps enable LOS and time-series alignment.
- Outcome markers are targets for modeling.
- Clinical numeric fields are the actual features used for vitals and labs.

**Why other fields are lower priority?**
- Billing or administrative fields often add noise and are not useful for physiological modeling.
- High-cardinality free-text fields require dedicated NLP preprocessing before use.


---

# Dataset: `icustays.csv`

**Goal:** Perform a clean EDA for `icustays.csv` following best-practice steps: setup & load, shape & dtypes, head/tail, missingness, descriptive stats, categorical unique counts, timestamp identification, and a short 'Required fields' analysis.


In [ ]:
# ---- EDA for icustays.csv
file_path = DATA_ROOT / 'icustays.csv'
if not file_path.exists():
    print('File not found:', file_path)
else:
    print('Processing file:', file_path)
    df = pd.read_csv(file_path, low_memory=False)
    print('\nShape:', df.shape)
    display(df.head())
    display(df.tail())
    # dtypes
    dtypes = df.dtypes.astype(str).rename('dtype')
    display(dtypes.to_frame())
    # missingness
    missing_counts = df.isna().sum().sort_values(ascending=False)
    missing_pct = (df.isna().mean()*100).sort_values(ascending=False)
    missing_summary = pd.DataFrame({'missing_count': missing_counts, 'missing_pct': missing_pct.round(2)})
    display(missing_summary)
    # descriptive stats
    try:
        display(df.describe(include='number').T)
    except Exception as e:
        print('Numeric describe error:', e)
    try:
        display(df.describe(include='object').T)
    except Exception as e:
        print('Object describe error:', e)
    # categorical unique counts
    cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
    if cat_cols:
        unique_summary = pd.DataFrame({c: [df[c].nunique()] for c in cat_cols}).T
        unique_summary.columns = ['n_unique']
        display(unique_summary)
    else:
        print('No categorical cols detected.')
    # timestamp detection (automatic heuristics + candidate names)
    import re
    parsed = {}
    for c in df.columns:
        try:
            _ = pd.to_datetime(df[c], errors='raise')
            parsed[c] = True
        except Exception:
            parsed[c] = False
    timestamp_cols = [c for c, ok in parsed.items() if ok]
    likely_time = re.compile(r'(time|date|charttime|storetime|admittime|dischtime|intime|outtime|edregtime|edouttime|deathtime)$', re.IGNORECASE)
    candidate_ts = [c for c in df.columns if likely_time.search(c)]
    print('\nAuto-parsed timestamp columns:', timestamp_cols)
    print('Candidate timestamp column names by pattern:', candidate_ts)
    for c in list(dict.fromkeys(timestamp_cols + candidate_ts)):
        s = pd.to_datetime(df[c], errors='coerce')
        print(c, '| min:', s.min(), '| max:', s.max(), '| #na:', s.isna().sum())
    # Quick correlations (if numeric columns < 100)
    numcols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numcols) > 1 and len(numcols) <= 100:
        corr = df[numcols].corr()
        print('\nNumeric columns correlation matrix (top-left 6x6 preview):')
        display(corr.iloc[:6, :6])
    else:
        print('\nSkipping correlation matrix (too many or too few numeric columns).')
    # Save a small sample CSV for quick inspection (first 100 rows)
    out_sample = DATA_ROOT.parent / 'Work' / 'samples'
    out_sample.mkdir(parents=True, exist_ok=True)
    sample_file = out_sample / 'icustays_sample_100.csv'
    df.head(100).to_csv(sample_file, index=False)
    print('Saved sample to:', sample_file)


Processing file: ..\Data\Raw_data\icustays.csv

Shape: (140, 8)


,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10018328,23786647,31269608,Neuro Stepdown,Neuro Stepdown,2154-04-24 23:03:44,2154-05-02 15:55:21,7.702512
1,10020187,24104168,37509585,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Stepdown,2169-01-15 04:56:00,2169-01-20 15:47:50,5.452662
2,10020187,26842957,32554129,Neuro Intermediate,Neuro Intermediate,2170-02-24 18:18:46,2170-02-25 15:15:26,0.872685
3,10012853,27882036,31338022,Trauma SICU (TSICU),Trauma SICU (TSICU),2176-11-26 02:34:49,2176-11-29 20:58:54,3.766725
4,10020740,25826145,32145159,Trauma SICU (TSICU),Trauma SICU (TSICU),2150-06-03 20:12:32,2150-06-04 21:05:58,1.037106


,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
135,10020786,23488445,33683112,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2189-06-09 12:46:30,2189-06-10 22:58:09,1.424757
136,10020740,23831430,35026312,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2150-03-11 15:34:56,2150-03-19 02:17:47,7.446424
137,10032725,20611640,30101877,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2143-03-22 06:42:00,2143-03-25 15:05:33,3.349687
138,10037928,24656677,39804682,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2178-12-21 06:05:18,2178-12-22 02:16:08,0.840856
139,10015931,22130791,37093652,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2177-03-24 21:48:07,2177-03-29 18:03:36,4.844086


,dtype
subject_id,int64
hadm_id,int64
stay_id,int64
first_careunit,object
last_careunit,object
intime,object
outtime,object
los,float64


,missing_count,missing_pct
subject_id,0,0.0
hadm_id,0,0.0
stay_id,0,0.0
first_careunit,0,0.0
last_careunit,0,0.0
intime,0,0.0
outtime,0,0.0
los,0,0.0


...

,n_unique
first_careunit,9
last_careunit,9
intime,140
outtime,140



Auto-parsed timestamp columns: ['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime', 'los']
Candidate timestamp column names by pattern: ['intime', 'outtime']
subject_id | min: 1970-01-01 00:00:00.010000032 | max: 1970-01-01 00:00:00.010040025 | #na: 0
hadm_id | min: 1970-01-01 00:00:00.020044587 | max: 1970-01-01 00:00:00.029974575 | #na: 0
stay_id | min: 1970-01-01 00:00:00.030057454 | max: 1970-01-01 00:00:00.039880770 | #na: 0
intime | min: 2110-04-11 15:52:22 | max: 2201-12-11 20:11:52 | #na: 0
outtime | min: 2110-04-12 23:59:56 | max: 2201-12-13 18:29:00 | #na: 0
los | min: 1970-01-01 00:00:00 | max: 1970-01-01 00:00:00.000000020 | #na: 0

Numeric columns correlation matrix (top-left 6x6 preview):


C:\Users\Adhis\AppData\Local\Temp\ipykernel_13632\858506041.py:41: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
C:\Users\Adhis\AppData\Local\Temp\ipykernel_13632\858506041.py:41: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
C:\Users\Adhis\AppData\Local\Temp\ipykernel_13632\858506041.py:41: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it i

,subject_id,hadm_id,stay_id,los
subject_id,1.000000,0.053632,0.055540,-0.012274
hadm_id,0.053632,1.000000,-0.053132,-0.019061
stay_id,0.055540,-0.053132,1.000000,-0.105860
los,-0.012274,-0.019061,-0.105860,1.000000


Saved sample to: ..\Data\Work\samples\icustays_sample_100.csv


### Required Fields Analysis for `icustays.csv`
Paste the output of the cell above and then fill the table below by hand or run automated reasoning prompts.

```
Required fields (suggested):
- Identifiers: subject_id, hadm_id, stay_id (if present)
- Timestamps: admittime, dischtime, intime, outtime, charttime, storetime, edregtime, edouttime
- Outcome markers: deathtime, hospital_expire_flag
- Clinical: for chartevents/labevents — itemid, value, valueuom, valuenum, charttime/storetime
- Metadata: admission_type, admission_location, discharge_location, insurance, ethnicity, language
```

**Why these fields?**
- Identifiers enable joins across datasets.
- Timestamps enable LOS and time-series alignment.
- Outcome markers are targets for modeling.
- Clinical numeric fields are the actual features used for vitals and labs.

**Why other fields are lower priority?**
- Billing or administrative fields often add noise and are not useful for physiological modeling.
- High-cardinality free-text fields require dedicated NLP preprocessing before use.


---

# Dataset: `chartevents.csv`

**Goal:** Perform a clean EDA for `chartevents.csv` following best-practice steps: setup & load, shape & dtypes, head/tail, missingness, descriptive stats, categorical unique counts, timestamp identification, and a short 'Required fields' analysis.


In [36]:
# ---- EDA for chartevents.csv
file_path = DATA_ROOT / 'chartevents.csv'
if not file_path.exists():
    print('File not found:', file_path)
else:
    print('Processing file:', file_path)
    df = pd.read_csv(file_path, low_memory=False)
    print('\nShape:', df.shape)
    display(df.head())
    display(df.tail())
    # dtypes
    dtypes = df.dtypes.astype(str).rename('dtype')
    display(dtypes.to_frame())
    # missingness
    missing_counts = df.isna().sum().sort_values(ascending=False)
    missing_pct = (df.isna().mean()*100).sort_values(ascending=False)
    missing_summary = pd.DataFrame({'missing_count': missing_counts, 'missing_pct': missing_pct.round(2)})
    display(missing_summary)
    # descriptive stats
    try:
        display(df.describe(include='number').T)
    except Exception as e:
        print('Numeric describe error:', e)
    try:
        display(df.describe(include='object').T)
    except Exception as e:
        print('Object describe error:', e)
    # categorical unique counts
    cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
    if cat_cols:
        unique_summary = pd.DataFrame({c: [df[c].nunique()] for c in cat_cols}).T
        unique_summary.columns = ['n_unique']
        display(unique_summary)
    else:
        print('No categorical cols detected.')
    # timestamp detection (automatic heuristics + candidate names)
    import re
    parsed = {}
    for c in df.columns:
        try:
            _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
            parsed[c] = True
        except Exception:
            parsed[c] = False
    timestamp_cols = [c for c, ok in parsed.items() if ok]
    likely_time = re.compile(r'(time|date|charttime|storetime|admittime|dischtime|intime|outtime|edregtime|edouttime|deathtime)$', re.IGNORECASE)
    candidate_ts = [c for c in df.columns if likely_time.search(c)]
    print('\nAuto-parsed timestamp columns:', timestamp_cols)
    print('Candidate timestamp column names by pattern:', candidate_ts)
    for c in list(dict.fromkeys(timestamp_cols + candidate_ts)):
        s = pd.to_datetime(df[c], errors='coerce')
        print(c, '| min:', s.min(), '| max:', s.max(), '| #na:', s.isna().sum())
    # Quick correlations (if numeric columns < 100)
    numcols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numcols) > 1 and len(numcols) <= 100:
        corr = df[numcols].corr()
        print('\nNumeric columns correlation matrix (top-left 6x6 preview):')
        display(corr.iloc[:6, :6])
    else:
        print('\nSkipping correlation matrix (too many or too few numeric columns).')
    # Save a small sample CSV for quick inspection (first 100 rows)
    out_sample = DATA_ROOT.parent / 'Work' / 'samples'
    out_sample.mkdir(parents=True, exist_ok=True)
    sample_file = out_sample / 'chartevents_sample_100.csv'
    df.head(100).to_csv(sample_file, index=False)
    print('Saved sample to:', sample_file)


Processing file: ..\Data\Raw_data\chartevents.csv

Shape: (668862, 11)

Shape: (668862, 11)


,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:45:00,225054,On,NaN,NaN,0.0
1,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:43:00,223769,100,100.0,%,0.0
2,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:47:00,223956,Atrial demand,NaN,NaN,0.0
3,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:47:00,224866,Yes,NaN,NaN,0.0
4,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:45:00,227341,No,0.0,NaN,0.0


,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
668857,10019003,29279905,34107647,60023.0,2153-03-28 10:49:28,NaN,220001,.Care Plan - Infection,NaN,NaN,NaN
668858,10019003,29279905,34107647,60023.0,2153-03-28 10:49:28,NaN,220001,"Diabetes Mellitus (DM), Type II",NaN,NaN,NaN
668859,10019003,29279905,34107647,60023.0,2153-03-28 10:49:28,NaN,220001,.Care Plan - Coping,NaN,NaN,NaN
668860,10019003,29279905,34107647,60023.0,2153-03-28 10:49:28,NaN,220001,"Chronic obstructive pulmonary disease (COPD, B...",NaN,NaN,NaN
668861,10019003,29279905,34107647,60023.0,2153-03-28 10:49:28,NaN,220001,.Care Plan - Alteration in Bowel Elimination: ...,NaN,NaN,NaN


,dtype
subject_id,int64
hadm_id,int64
stay_id,int64
caregiver_id,float64
charttime,object
storetime,object
itemid,int64
value,object
valuenum,float64
valueuom,object


,missing_count,missing_pct
valueuom,506291,75.69
valuenum,411388,61.51
caregiver_id,24240,3.62
value,20730,3.10
storetime,1159,0.17
warning,1159,0.17
subject_id,0,0.00
stay_id,0,0.00
hadm_id,0,0.00
itemid,0,0.00


...

,n_unique
charttime,36303
storetime,53751
value,4241
valueuom,43


C:\Users\Adhis\AppData\Local\Temp\ipykernel_13632\1475032673.py:41: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
C:\Users\Adhis\AppData\Local\Temp\ipykernel_13632\1475032673.py:41: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
C:\Users\Adhis\AppData\Local\Temp\ipykernel_13632\1475032673.py:41: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of i


Auto-parsed timestamp columns: ['subject_id', 'hadm_id', 'stay_id', 'caregiver_id', 'charttime', 'storetime', 'itemid', 'valuenum', 'warning']
Candidate timestamp column names by pattern: ['charttime', 'storetime']
subject_id | min: 1970-01-01 00:00:00.010000032 | max: 1970-01-01 00:00:00.010040025 | #na: 0
hadm_id | min: 1970-01-01 00:00:00.020044587 | max: 1970-01-01 00:00:00.029974575 | #na: 0
stay_id | min: 1970-01-01 00:00:00.030057454 | max: 1970-01-01 00:00:00.039880770 | #na: 0
caregiver_id | min: 1970-01-01 00:00:00.000000199 | max: 1970-01-01 00:00:00.000099850 | #na: 24240
charttime | min: 2110-04-11 15:52:00 | max: 2201-12-13 18:16:00 | #na: 0
storetime | min: 2110-04-11 16:01:00 | max: 2201-12-13 18:06:00 | #na: 1159
itemid | min: 1970-01-01 00:00:00.000220001 | max: 1970-01-01 00:00:00.000229858 | #na: 0
charttime | min: 2110-04-11 15:52:00 | max: 2201-12-13 18:16:00 | #na: 0
storetime | min: 2110-04-11 16:01:00 | max: 2201-12-13 18:06:00 | #na: 1159
itemid | min: 1970-0

,subject_id,hadm_id,stay_id,caregiver_id,itemid,valuenum
subject_id,1.000000,0.287165,-0.041735,-0.049383,-0.002031,0.003030
hadm_id,0.287165,1.000000,-0.121471,0.027237,-0.019174,0.001307
stay_id,-0.041735,-0.121471,1.000000,0.001404,0.012877,-0.002557
caregiver_id,-0.049383,0.027237,0.001404,1.000000,-0.006481,0.002713
itemid,-0.002031,-0.019174,0.012877,-0.006481,1.000000,0.000289
valuenum,0.003030,0.001307,-0.002557,0.002713,0.000289,1.000000


Saved sample to: ..\Data\Work\samples\chartevents_sample_100.csv


### Required Fields Analysis for `chartevents.csv`
Paste the output of the cell above and then fill the table below by hand or run automated reasoning prompts.

```
Required fields (suggested):
- Identifiers: subject_id, hadm_id, stay_id (if present)
- Timestamps: admittime, dischtime, intime, outtime, charttime, storetime, edregtime, edouttime
- Outcome markers: deathtime, hospital_expire_flag
- Clinical: for chartevents/labevents — itemid, value, valueuom, valuenum, charttime/storetime
- Metadata: admission_type, admission_location, discharge_location, insurance, ethnicity, language
```

**Why these fields?**
- Identifiers enable joins across datasets.
- Timestamps enable LOS and time-series alignment.
- Outcome markers are targets for modeling.
- Clinical numeric fields are the actual features used for vitals and labs.

**Why other fields are lower priority?**
- Billing or administrative fields often add noise and are not useful for physiological modeling.
- High-cardinality free-text fields require dedicated NLP preprocessing before use.


---

# Dataset: `labevents.csv`

**Goal:** Perform a clean EDA for `labevents.csv` following best-practice steps: setup & load, shape & dtypes, head/tail, missingness, descriptive stats, categorical unique counts, timestamp identification, and a short 'Required fields' analysis.


In [37]:
# ---- EDA for labevents.csv
file_path = DATA_ROOT / 'labevents.csv'
if not file_path.exists():
    print('File not found:', file_path)
else:
    print('Processing file:', file_path)
    df = pd.read_csv(file_path, low_memory=False)
    print('\nShape:', df.shape)
    display(df.head())
    display(df.tail())
    # dtypes
    dtypes = df.dtypes.astype(str).rename('dtype')
    display(dtypes.to_frame())
    # missingness
    missing_counts = df.isna().sum().sort_values(ascending=False)
    missing_pct = (df.isna().mean()*100).sort_values(ascending=False)
    missing_summary = pd.DataFrame({'missing_count': missing_counts, 'missing_pct': missing_pct.round(2)})
    display(missing_summary)
    # descriptive stats
    try:
        display(df.describe(include='number').T)
    except Exception as e:
        print('Numeric describe error:', e)
    try:
        display(df.describe(include='object').T)
    except Exception as e:
        print('Object describe error:', e)
    # categorical unique counts
    cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
    if cat_cols:
        unique_summary = pd.DataFrame({c: [df[c].nunique()] for c in cat_cols}).T
        unique_summary.columns = ['n_unique']
        display(unique_summary)
    else:
        print('No categorical cols detected.')
    # timestamp detection (automatic heuristics + candidate names)
    import re
    parsed = {}
    for c in df.columns:
        try:
            _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
            parsed[c] = True
        except Exception:
            parsed[c] = False
    timestamp_cols = [c for c, ok in parsed.items() if ok]
    likely_time = re.compile(r'(time|date|charttime|storetime|admittime|dischtime|intime|outtime|edregtime|edouttime|deathtime)$', re.IGNORECASE)
    candidate_ts = [c for c in df.columns if likely_time.search(c)]
    print('\nAuto-parsed timestamp columns:', timestamp_cols)
    print('Candidate timestamp column names by pattern:', candidate_ts)
    for c in list(dict.fromkeys(timestamp_cols + candidate_ts)):
        s = pd.to_datetime(df[c], errors='coerce')
        print(c, '| min:', s.min(), '| max:', s.max(), '| #na:', s.isna().sum())
    # Quick correlations (if numeric columns < 100)
    numcols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numcols) > 1 and len(numcols) <= 100:
        corr = df[numcols].corr()
        print('\nNumeric columns correlation matrix (top-left 6x6 preview):')
        display(corr.iloc[:6, :6])
    else:
        print('\nSkipping correlation matrix (too many or too few numeric columns).')
    # Save a small sample CSV for quick inspection (first 100 rows)
    out_sample = DATA_ROOT.parent / 'Work' / 'samples'
    out_sample.mkdir(parents=True, exist_ok=True)
    sample_file = out_sample / 'labevents_sample_100.csv'
    df.head(100).to_csv(sample_file, index=False)
    print('Saved sample to:', sample_file)


Processing file: ..\Data\Raw_data\labevents.csv

Shape: (107727, 16)

Shape: (107727, 16)


,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,172061,10014354,29600294.0,1808066,51277,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,15.4,15.40,%,10.5,15.5,NaN,ROUTINE,NaN
1,172062,10014354,29600294.0,1808066,51279,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,3.35,3.35,m/uL,4.6,6.1,abnormal,ROUTINE,NaN
2,172068,10014354,29600294.0,1808066,52172,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,49.7,49.70,fL,35.1,46.3,abnormal,ROUTINE,NaN
3,172063,10014354,29600294.0,1808066,51301,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,20.3,20.30,K/uL,4.0,10.0,abnormal,ROUTINE,NaN
4,172050,10014354,29600294.0,1808066,51249,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,31.1,31.10,g/dL,32.0,37.0,abnormal,ROUTINE,NaN


,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
107722,254700,10021487,28998349.0,78791160,50804,NaN,2116-12-07 18:59:00,2116-12-07 19:00:00,35,35.0,mEq/L,21.0,30.0,abnormal,NaN,NaN
107723,254702,10021487,28998349.0,78791160,50818,NaN,2116-12-07 18:59:00,2116-12-07 19:00:00,56,56.0,mm Hg,35.0,45.0,abnormal,NaN,NaN
107724,254707,10021487,28998349.0,78791160,52033,NaN,2116-12-07 18:59:00,2116-12-07 18:59:00,ART.,NaN,NaN,NaN,NaN,NaN,NaN,___
107725,254706,10021487,28998349.0,78791160,50825,NaN,2116-12-07 18:59:00,2116-12-07 18:59:00,39.7,39.7,NaN,NaN,NaN,NaN,NaN,NaN
107726,454886,10038999,27189241.0,13534028,50873,NaN,2131-05-23 21:59:00,2131-05-25 14:36:00,___,NaN,NaN,NaN,NaN,NaN,STAT,NEGATIVE.


,dtype
labevent_id,int64
subject_id,int64
hadm_id,float64
specimen_id,int64
itemid,int64
order_provider_id,object
charttime,object
storetime,object
value,object
valuenum,float64


,missing_count,missing_pct
order_provider_id,90897,84.38
comments,89273,82.87
flag,67452,62.61
hadm_id,28420,26.38
ref_range_upper,18728,17.38
ref_range_lower,18728,17.38
valueuom,16203,15.04
valuenum,12481,11.59
value,9588,8.90
priority,9329,8.66


...

,n_unique
order_provider_id,151
charttime,5864
storetime,14573
value,3107
valueuom,49
flag,1
priority,2
comments,736


C:\Users\Adhis\AppData\Local\Temp\ipykernel_13632\2241675839.py:41: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
C:\Users\Adhis\AppData\Local\Temp\ipykernel_13632\2241675839.py:41: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
C:\Users\Adhis\AppData\Local\Temp\ipykernel_13632\2241675839.py:41: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of i


Auto-parsed timestamp columns: ['labevent_id', 'subject_id', 'hadm_id', 'specimen_id', 'itemid', 'charttime', 'storetime', 'valuenum', 'ref_range_lower', 'ref_range_upper']
Candidate timestamp column names by pattern: ['charttime', 'storetime']
labevent_id | min: 1970-01-01 00:00:00.000000001 | max: 1970-01-01 00:00:00.000474464 | #na: 0
subject_id | min: 1970-01-01 00:00:00.010000032 | max: 1970-01-01 00:00:00.010040025 | #na: 0
hadm_id | min: 1970-01-01 00:00:00.020044587 | max: 1970-01-01 00:00:00.029974575 | #na: 28420
specimen_id | min: 1970-01-01 00:00:00.000000849 | max: 1970-01-01 00:00:00.099985495 | #na: 0
itemid | min: 1970-01-01 00:00:00.000050801 | max: 1970-01-01 00:00:00.000053153 | #na: 0
charttime | min: 2110-04-08 11:11:00 | max: 2202-08-26 14:00:00 | #na: 0
storetime | min: 2110-04-08 17:49:00 | max: 2202-08-26 14:13:00 | #na: 992
valuenum | min: 1969-12-31 23:59:59.999999220 | max: 1970-01-01 00:00:00.000169 | #na: 12481
ref_range_lower | min: 1970-01-01 00:00:00 |

,labevent_id,subject_id,hadm_id,specimen_id,itemid,valuenum
labevent_id,1.000000,0.999434,0.109061,-0.002872,0.001324,-0.000741
subject_id,0.999434,1.000000,0.109930,-0.002423,-0.000227,-0.000496
hadm_id,0.109061,0.109930,1.000000,-0.014226,0.006083,0.001669
specimen_id,-0.002872,-0.002423,-0.014226,1.000000,-0.003052,-0.002303
itemid,0.001324,-0.000227,0.006083,-0.003052,1.000000,-0.006664
valuenum,-0.000741,-0.000496,0.001669,-0.002303,-0.006664,1.000000


Saved sample to: ..\Data\Work\samples\labevents_sample_100.csv


### Required Fields Analysis for `labevents.csv`
Paste the output of the cell above and then fill the table below by hand or run automated reasoning prompts.

```
Required fields (suggested):
- Identifiers: subject_id, hadm_id, stay_id (if present)
- Timestamps: admittime, dischtime, intime, outtime, charttime, storetime, edregtime, edouttime
- Outcome markers: deathtime, hospital_expire_flag
- Clinical: for chartevents/labevents — itemid, value, valueuom, valuenum, charttime/storetime
- Metadata: admission_type, admission_location, discharge_location, insurance, ethnicity, language
```

**Why these fields?**
- Identifiers enable joins across datasets.
- Timestamps enable LOS and time-series alignment.
- Outcome markers are targets for modeling.
- Clinical numeric fields are the actual features used for vitals and labs.

**Why other fields are lower priority?**
- Billing or administrative fields often add noise and are not useful for physiological modeling.
- High-cardinality free-text fields require dedicated NLP preprocessing before use.


---

# Dataset: `d_labitems.csv`

**Goal:** Perform a clean EDA for `d_labitems.csv` following best-practice steps: setup & load, shape & dtypes, head/tail, missingness, descriptive stats, categorical unique counts, timestamp identification, and a short 'Required fields' analysis.


In [23]:
# ---- EDA for d_labitems.csv
file_path = DATA_ROOT / 'd_labitems.csv'
if not file_path.exists():
    print('File not found:', file_path)
else:
    print('Processing file:', file_path)
    df = pd.read_csv(file_path, low_memory=False)
    print('\nShape:', df.shape)
    display(df.head())
    display(df.tail())
    # dtypes
    dtypes = df.dtypes.astype(str).rename('dtype')
    display(dtypes.to_frame())
    # missingness
    missing_counts = df.isna().sum().sort_values(ascending=False)
    missing_pct = (df.isna().mean()*100).sort_values(ascending=False)
    missing_summary = pd.DataFrame({'missing_count': missing_counts, 'missing_pct': missing_pct.round(2)})
    display(missing_summary)
    # descriptive stats
    try:
        display(df.describe(include='number').T)
    except Exception as e:
        print('Numeric describe error:', e)
    try:
        display(df.describe(include='object').T)
    except Exception as e:
        print('Object describe error:', e)
    # categorical unique counts
    cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
    if cat_cols:
        unique_summary = pd.DataFrame({c: [df[c].nunique()] for c in cat_cols}).T
        unique_summary.columns = ['n_unique']
        display(unique_summary)
    else:
        print('No categorical cols detected.')
    # timestamp detection (automatic heuristics + candidate names)
    import re
    parsed = {}
    for c in df.columns:
        try:
            _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
            parsed[c] = True
        except Exception:
            parsed[c] = False
    timestamp_cols = [c for c, ok in parsed.items() if ok]
    likely_time = re.compile(r'(time|date|charttime|storetime|admittime|dischtime|intime|outtime|edregtime|edouttime|deathtime)$', re.IGNORECASE)
    candidate_ts = [c for c in df.columns if likely_time.search(c)]
    print('\nAuto-parsed timestamp columns:', timestamp_cols)
    print('Candidate timestamp column names by pattern:', candidate_ts)
    for c in list(dict.fromkeys(timestamp_cols + candidate_ts)):
        s = pd.to_datetime(df[c], errors='coerce')
        print(c, '| min:', s.min(), '| max:', s.max(), '| #na:', s.isna().sum())
    # Quick correlations (if numeric columns < 100)
    numcols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numcols) > 1 and len(numcols) <= 100:
        corr = df[numcols].corr()
        print('\nNumeric columns correlation matrix (top-left 6x6 preview):')
        display(corr.iloc[:6, :6])
    else:
        print('\nSkipping correlation matrix (too many or too few numeric columns).')
    # Save a small sample CSV for quick inspection (first 100 rows)
    out_sample = DATA_ROOT.parent / 'Work' / 'samples'
    out_sample.mkdir(parents=True, exist_ok=True)
    sample_file = out_sample / 'd_labitems_sample_100.csv'
    df.head(100).to_csv(sample_file, index=False)
    print('Saved sample to:', sample_file)


File not found: Data\Raw_data\d_labitems.csv


### Required Fields Analysis for `d_labitems.csv`
Paste the output of the cell above and then fill the table below by hand or run automated reasoning prompts.

```
Required fields (suggested):
- Identifiers: subject_id, hadm_id, stay_id (if present)
- Timestamps: admittime, dischtime, intime, outtime, charttime, storetime, edregtime, edouttime
- Outcome markers: deathtime, hospital_expire_flag
- Clinical: for chartevents/labevents — itemid, value, valueuom, valuenum, charttime/storetime
- Metadata: admission_type, admission_location, discharge_location, insurance, ethnicity, language
```

**Why these fields?**
- Identifiers enable joins across datasets.
- Timestamps enable LOS and time-series alignment.
- Outcome markers are targets for modeling.
- Clinical numeric fields are the actual features used for vitals and labs.

**Why other fields are lower priority?**
- Billing or administrative fields often add noise and are not useful for physiological modeling.
- High-cardinality free-text fields require dedicated NLP preprocessing before use.


---

# Dataset: `d_items.csv`

**Goal:** Perform a clean EDA for `d_items.csv` following best-practice steps: setup & load, shape & dtypes, head/tail, missingness, descriptive stats, categorical unique counts, timestamp identification, and a short 'Required fields' analysis.


In [24]:
# ---- EDA for d_items.csv
file_path = DATA_ROOT / 'd_items.csv'
if not file_path.exists():
    print('File not found:', file_path)
else:
    print('Processing file:', file_path)
    df = pd.read_csv(file_path, low_memory=False)
    print('\nShape:', df.shape)
    display(df.head())
    display(df.tail())
    # dtypes
    dtypes = df.dtypes.astype(str).rename('dtype')
    display(dtypes.to_frame())
    # missingness
    missing_counts = df.isna().sum().sort_values(ascending=False)
    missing_pct = (df.isna().mean()*100).sort_values(ascending=False)
    missing_summary = pd.DataFrame({'missing_count': missing_counts, 'missing_pct': missing_pct.round(2)})
    display(missing_summary)
    # descriptive stats
    try:
        display(df.describe(include='number').T)
    except Exception as e:
        print('Numeric describe error:', e)
    try:
        display(df.describe(include='object').T)
    except Exception as e:
        print('Object describe error:', e)
    # categorical unique counts
    cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
    if cat_cols:
        unique_summary = pd.DataFrame({c: [df[c].nunique()] for c in cat_cols}).T
        unique_summary.columns = ['n_unique']
        display(unique_summary)
    else:
        print('No categorical cols detected.')
    # timestamp detection (automatic heuristics + candidate names)
    import re
    parsed = {}
    for c in df.columns:
        try:
            _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
            parsed[c] = True
        except Exception:
            parsed[c] = False
    timestamp_cols = [c for c, ok in parsed.items() if ok]
    likely_time = re.compile(r'(time|date|charttime|storetime|admittime|dischtime|intime|outtime|edregtime|edouttime|deathtime)$', re.IGNORECASE)
    candidate_ts = [c for c in df.columns if likely_time.search(c)]
    print('\nAuto-parsed timestamp columns:', timestamp_cols)
    print('Candidate timestamp column names by pattern:', candidate_ts)
    for c in list(dict.fromkeys(timestamp_cols + candidate_ts)):
        s = pd.to_datetime(df[c], errors='coerce')
        print(c, '| min:', s.min(), '| max:', s.max(), '| #na:', s.isna().sum())
    # Quick correlations (if numeric columns < 100)
    numcols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numcols) > 1 and len(numcols) <= 100:
        corr = df[numcols].corr()
        print('\nNumeric columns correlation matrix (top-left 6x6 preview):')
        display(corr.iloc[:6, :6])
    else:
        print('\nSkipping correlation matrix (too many or too few numeric columns).')
    # Save a small sample CSV for quick inspection (first 100 rows)
    out_sample = DATA_ROOT.parent / 'Work' / 'samples'
    out_sample.mkdir(parents=True, exist_ok=True)
    sample_file = out_sample / 'd_items_sample_100.csv'
    df.head(100).to_csv(sample_file, index=False)
    print('Saved sample to:', sample_file)


File not found: Data\Raw_data\d_items.csv


### Required Fields Analysis for `d_items.csv`
Paste the output of the cell above and then fill the table below by hand or run automated reasoning prompts.

```
Required fields (suggested):
- Identifiers: subject_id, hadm_id, stay_id (if present)
- Timestamps: admittime, dischtime, intime, outtime, charttime, storetime, edregtime, edouttime
- Outcome markers: deathtime, hospital_expire_flag
- Clinical: for chartevents/labevents — itemid, value, valueuom, valuenum, charttime/storetime
- Metadata: admission_type, admission_location, discharge_location, insurance, ethnicity, language
```

**Why these fields?**
- Identifiers enable joins across datasets.
- Timestamps enable LOS and time-series alignment.
- Outcome markers are targets for modeling.
- Clinical numeric fields are the actual features used for vitals and labs.

**Why other fields are lower priority?**
- Billing or administrative fields often add noise and are not useful for physiological modeling.
- High-cardinality free-text fields require dedicated NLP preprocessing before use.


---

# Dataset: `omr.csv`

**Goal:** Perform a clean EDA for `omr.csv` following best-practice steps: setup & load, shape & dtypes, head/tail, missingness, descriptive stats, categorical unique counts, timestamp identification, and a short 'Required fields' analysis.


In [52]:
# ---- EDA for omr.csv
file_path = DATA_ROOT / 'omr.csv'
if not file_path.exists():
    print('File not found:', file_path)
else:
    print('Processing file:', file_path)
    df = pd.read_csv(file_path, low_memory=False)
    print('\nShape:', df.shape)
    display(df.head())
    display(df.tail())
    # dtypes
    dtypes = df.dtypes.astype(str).rename('dtype')
    display(dtypes.to_frame())
    # missingness
    missing_counts = df.isna().sum().sort_values(ascending=False)
    missing_pct = (df.isna().mean()*100).sort_values(ascending=False)
    missing_summary = pd.DataFrame({'missing_count': missing_counts, 'missing_pct': missing_pct.round(2)})
    display(missing_summary)
    # descriptive stats
    try:
        display(df.describe(include='number').T)
    except Exception as e:
        print('Numeric describe error:', e)
    try:
        display(df.describe(include='object').T)
    except Exception as e:
        print('Object describe error:', e)
    # categorical unique counts
    cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
    if cat_cols:
        unique_summary = pd.DataFrame({c: [df[c].nunique()] for c in cat_cols}).T
        unique_summary.columns = ['n_unique']
        display(unique_summary)
    else:
        print('No categorical cols detected.')
    # timestamp detection (automatic heuristics + candidate names)
    import re
    parsed = {}
    for c in df.columns:
        try:
            _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
            parsed[c] = True
        except Exception:
            parsed[c] = False
    timestamp_cols = [c for c, ok in parsed.items() if ok]
    likely_time = re.compile(r'(time|date|charttime|storetime|admittime|dischtime|intime|outtime|edregtime|edouttime|deathtime)$', re.IGNORECASE)
    candidate_ts = [c for c in df.columns if likely_time.search(c)]
    print('\nAuto-parsed timestamp columns:', timestamp_cols)
    print('Candidate timestamp column names by pattern:', candidate_ts)
    for c in list(dict.fromkeys(timestamp_cols + candidate_ts)):
        s = pd.to_datetime(df[c], errors='coerce')
        print(c, '| min:', s.min(), '| max:', s.max(), '| #na:', s.isna().sum())
    # Quick correlations (if numeric columns < 100)
    numcols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numcols) > 1 and len(numcols) <= 100:
        corr = df[numcols].corr()
        print('\nNumeric columns correlation matrix (top-left 6x6 preview):')
        display(corr.iloc[:6, :6])
    else:
        print('\nSkipping correlation matrix (too many or too few numeric columns).')
    # Save a small sample CSV for quick inspection (first 100 rows)
    out_sample = DATA_ROOT.parent / 'Work' / 'samples'
    out_sample.mkdir(parents=True, exist_ok=True)
    sample_file = out_sample / 'omr_sample_100.csv'
    df.head(100).to_csv(sample_file, index=False)
    print('Saved sample to:', sample_file)


Processing file: ..\Data\Raw_data\omr.csv

Shape: (2964, 5)

Shape: (2964, 5)


,subject_id,chartdate,seq_num,result_name,result_value
0,10011398,2146-12-01,1,Height (Inches),63
1,10011398,2147-01-22,1,Weight (Lbs),127
2,10011398,2146-12-01,1,Weight (Lbs),135
3,10011398,2147-07-24,1,Weight (Lbs),136
4,10011398,2147-03-26,1,Weight (Lbs),136


,subject_id,chartdate,seq_num,result_name,result_value
2959,10021118,2161-11-15,1,Height (Inches),71
2960,10021118,2161-11-15,1,BMI (kg/m2),35.2
2961,10021118,2161-11-29,1,Blood Pressure,111/72
2962,10021118,2161-12-25,1,Blood Pressure,135/76
2963,10021118,2161-11-15,1,Weight (Lbs),252.65


,dtype
subject_id,int64
chartdate,object
seq_num,int64
result_name,object
result_value,object


,missing_count,missing_pct
subject_id,0,0.0
chartdate,0,0.0
seq_num,0,0.0
result_name,0,0.0
result_value,0,0.0


...

,n_unique
chartdate,945
result_name,8
result_value,1427


C:\Users\Adhis\AppData\Local\Temp\ipykernel_13632\460987798.py:41: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
C:\Users\Adhis\AppData\Local\Temp\ipykernel_13632\460987798.py:41: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
C:\Users\Adhis\AppData\Local\Temp\ipykernel_13632\460987798.py:41: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it i


Auto-parsed timestamp columns: ['subject_id', 'chartdate', 'seq_num']
Candidate timestamp column names by pattern: ['chartdate']
subject_id | min: 1970-01-01 00:00:00.010000032 | max: 1970-01-01 00:00:00.010040025 | #na: 0
chartdate | min: 2109-10-15 00:00:00 | max: 2202-12-17 00:00:00 | #na: 0
seq_num | min: 1970-01-01 00:00:00.000000001 | max: 1970-01-01 00:00:00.000000027 | #na: 0

Numeric columns correlation matrix (top-left 6x6 preview):


,subject_id,seq_num
subject_id,1.000000,0.050249
seq_num,0.050249,1.000000


Saved sample to: ..\Data\Work\samples\omr_sample_100.csv


### Required Fields Analysis for `omr.csv`
Paste the output of the cell above and then fill the table below by hand or run automated reasoning prompts.

```
Required fields (suggested):
- Identifiers: subject_id, hadm_id, stay_id (if present)
- Timestamps: admittime, dischtime, intime, outtime, charttime, storetime, edregtime, edouttime
- Outcome markers: deathtime, hospital_expire_flag
- Clinical: for chartevents/labevents — itemid, value, valueuom, valuenum, charttime/storetime
- Metadata: admission_type, admission_location, discharge_location, insurance, ethnicity, language
```

**Why these fields?**
- Identifiers enable joins across datasets.
- Timestamps enable LOS and time-series alignment.
- Outcome markers are targets for modeling.
- Clinical numeric fields are the actual features used for vitals and labs.

**Why other fields are lower priority?**
- Billing or administrative fields often add noise and are not useful for physiological modeling.
- High-cardinality free-text fields require dedicated NLP preprocessing before use.


---

# Dataset: `prescriptions.csv`

**Goal:** Perform a clean EDA for `prescriptions.csv` following best-practice steps: setup & load, shape & dtypes, head/tail, missingness, descriptive stats, categorical unique counts, timestamp identification, and a short 'Required fields' analysis.


In [26]:
# ---- EDA for prescriptions.csv
file_path = DATA_ROOT / 'prescriptions.csv'
if not file_path.exists():
    print('File not found:', file_path)
else:
    print('Processing file:', file_path)
    df = pd.read_csv(file_path, low_memory=False)
    print('\nShape:', df.shape)
    display(df.head())
    display(df.tail())
    # dtypes
    dtypes = df.dtypes.astype(str).rename('dtype')
    display(dtypes.to_frame())
    # missingness
    missing_counts = df.isna().sum().sort_values(ascending=False)
    missing_pct = (df.isna().mean()*100).sort_values(ascending=False)
    missing_summary = pd.DataFrame({'missing_count': missing_counts, 'missing_pct': missing_pct.round(2)})
    display(missing_summary)
    # descriptive stats
    try:
        display(df.describe(include='number').T)
    except Exception as e:
        print('Numeric describe error:', e)
    try:
        display(df.describe(include='object').T)
    except Exception as e:
        print('Object describe error:', e)
    # categorical unique counts
    cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
    if cat_cols:
        unique_summary = pd.DataFrame({c: [df[c].nunique()] for c in cat_cols}).T
        unique_summary.columns = ['n_unique']
        display(unique_summary)
    else:
        print('No categorical cols detected.')
    # timestamp detection (automatic heuristics + candidate names)
    import re
    parsed = {}
    for c in df.columns:
        try:
            _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
            parsed[c] = True
        except Exception:
            parsed[c] = False
    timestamp_cols = [c for c, ok in parsed.items() if ok]
    likely_time = re.compile(r'(time|date|charttime|storetime|admittime|dischtime|intime|outtime|edregtime|edouttime|deathtime)$', re.IGNORECASE)
    candidate_ts = [c for c in df.columns if likely_time.search(c)]
    print('\nAuto-parsed timestamp columns:', timestamp_cols)
    print('Candidate timestamp column names by pattern:', candidate_ts)
    for c in list(dict.fromkeys(timestamp_cols + candidate_ts)):
        s = pd.to_datetime(df[c], errors='coerce')
        print(c, '| min:', s.min(), '| max:', s.max(), '| #na:', s.isna().sum())
    # Quick correlations (if numeric columns < 100)
    numcols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numcols) > 1 and len(numcols) <= 100:
        corr = df[numcols].corr()
        print('\nNumeric columns correlation matrix (top-left 6x6 preview):')
        display(corr.iloc[:6, :6])
    else:
        print('\nSkipping correlation matrix (too many or too few numeric columns).')
    # Save a small sample CSV for quick inspection (first 100 rows)
    out_sample = DATA_ROOT.parent / 'Work' / 'samples'
    out_sample.mkdir(parents=True, exist_ok=True)
    sample_file = out_sample / 'prescriptions_sample_100.csv'
    df.head(100).to_csv(sample_file, index=False)
    print('Saved sample to:', sample_file)


File not found: Data\Raw_data\prescriptions.csv


### Required Fields Analysis for `prescriptions.csv`
Paste the output of the cell above and then fill the table below by hand or run automated reasoning prompts.

```
Required fields (suggested):
- Identifiers: subject_id, hadm_id, stay_id (if present)
- Timestamps: admittime, dischtime, intime, outtime, charttime, storetime, edregtime, edouttime
- Outcome markers: deathtime, hospital_expire_flag
- Clinical: for chartevents/labevents — itemid, value, valueuom, valuenum, charttime/storetime
- Metadata: admission_type, admission_location, discharge_location, insurance, ethnicity, language
```

**Why these fields?**
- Identifiers enable joins across datasets.
- Timestamps enable LOS and time-series alignment.
- Outcome markers are targets for modeling.
- Clinical numeric fields are the actual features used for vitals and labs.

**Why other fields are lower priority?**
- Billing or administrative fields often add noise and are not useful for physiological modeling.
- High-cardinality free-text fields require dedicated NLP preprocessing before use.


---

# Dataset: `outputevents.csv`

**Goal:** Perform a clean EDA for `outputevents.csv` following best-practice steps: setup & load, shape & dtypes, head/tail, missingness, descriptive stats, categorical unique counts, timestamp identification, and a short 'Required fields' analysis.


In [27]:
# ---- EDA for outputevents.csv
file_path = DATA_ROOT / 'outputevents.csv'
if not file_path.exists():
    print('File not found:', file_path)
else:
    print('Processing file:', file_path)
    df = pd.read_csv(file_path, low_memory=False)
    print('\nShape:', df.shape)
    display(df.head())
    display(df.tail())
    # dtypes
    dtypes = df.dtypes.astype(str).rename('dtype')
    display(dtypes.to_frame())
    # missingness
    missing_counts = df.isna().sum().sort_values(ascending=False)
    missing_pct = (df.isna().mean()*100).sort_values(ascending=False)
    missing_summary = pd.DataFrame({'missing_count': missing_counts, 'missing_pct': missing_pct.round(2)})
    display(missing_summary)
    # descriptive stats
    try:
        display(df.describe(include='number').T)
    except Exception as e:
        print('Numeric describe error:', e)
    try:
        display(df.describe(include='object').T)
    except Exception as e:
        print('Object describe error:', e)
    # categorical unique counts
    cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
    if cat_cols:
        unique_summary = pd.DataFrame({c: [df[c].nunique()] for c in cat_cols}).T
        unique_summary.columns = ['n_unique']
        display(unique_summary)
    else:
        print('No categorical cols detected.')
    # timestamp detection (automatic heuristics + candidate names)
    import re
    parsed = {}
    for c in df.columns:
        try:
            _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
            parsed[c] = True
        except Exception:
            parsed[c] = False
    timestamp_cols = [c for c, ok in parsed.items() if ok]
    likely_time = re.compile(r'(time|date|charttime|storetime|admittime|dischtime|intime|outtime|edregtime|edouttime|deathtime)$', re.IGNORECASE)
    candidate_ts = [c for c in df.columns if likely_time.search(c)]
    print('\nAuto-parsed timestamp columns:', timestamp_cols)
    print('Candidate timestamp column names by pattern:', candidate_ts)
    for c in list(dict.fromkeys(timestamp_cols + candidate_ts)):
        s = pd.to_datetime(df[c], errors='coerce')
        print(c, '| min:', s.min(), '| max:', s.max(), '| #na:', s.isna().sum())
    # Quick correlations (if numeric columns < 100)
    numcols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numcols) > 1 and len(numcols) <= 100:
        corr = df[numcols].corr()
        print('\nNumeric columns correlation matrix (top-left 6x6 preview):')
        display(corr.iloc[:6, :6])
    else:
        print('\nSkipping correlation matrix (too many or too few numeric columns).')
    # Save a small sample CSV for quick inspection (first 100 rows)
    out_sample = DATA_ROOT.parent / 'Work' / 'samples'
    out_sample.mkdir(parents=True, exist_ok=True)
    sample_file = out_sample / 'outputevents_sample_100.csv'
    df.head(100).to_csv(sample_file, index=False)
    print('Saved sample to:', sample_file)


File not found: Data\Raw_data\outputevents.csv


### Required Fields Analysis for `outputevents.csv`
Paste the output of the cell above and then fill the table below by hand or run automated reasoning prompts.

```
Required fields (suggested):
- Identifiers: subject_id, hadm_id, stay_id (if present)
- Timestamps: admittime, dischtime, intime, outtime, charttime, storetime, edregtime, edouttime
- Outcome markers: deathtime, hospital_expire_flag
- Clinical: for chartevents/labevents — itemid, value, valueuom, valuenum, charttime/storetime
- Metadata: admission_type, admission_location, discharge_location, insurance, ethnicity, language
```

**Why these fields?**
- Identifiers enable joins across datasets.
- Timestamps enable LOS and time-series alignment.
- Outcome markers are targets for modeling.
- Clinical numeric fields are the actual features used for vitals and labs.

**Why other fields are lower priority?**
- Billing or administrative fields often add noise and are not useful for physiological modeling.
- High-cardinality free-text fields require dedicated NLP preprocessing before use.


---

# Dataset: `emar.csv`

**Goal:** Perform a clean EDA for `emar.csv` following best-practice steps: setup & load, shape & dtypes, head/tail, missingness, descriptive stats, categorical unique counts, timestamp identification, and a short 'Required fields' analysis.


In [28]:
# ---- EDA for emar.csv
file_path = DATA_ROOT / 'emar.csv'
if not file_path.exists():
    print('File not found:', file_path)
else:
    print('Processing file:', file_path)
    df = pd.read_csv(file_path, low_memory=False)
    print('\nShape:', df.shape)
    display(df.head())
    display(df.tail())
    # dtypes
    dtypes = df.dtypes.astype(str).rename('dtype')
    display(dtypes.to_frame())
    # missingness
    missing_counts = df.isna().sum().sort_values(ascending=False)
    missing_pct = (df.isna().mean()*100).sort_values(ascending=False)
    missing_summary = pd.DataFrame({'missing_count': missing_counts, 'missing_pct': missing_pct.round(2)})
    display(missing_summary)
    # descriptive stats
    try:
        display(df.describe(include='number').T)
    except Exception as e:
        print('Numeric describe error:', e)
    try:
        display(df.describe(include='object').T)
    except Exception as e:
        print('Object describe error:', e)
    # categorical unique counts
    cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
    if cat_cols:
        unique_summary = pd.DataFrame({c: [df[c].nunique()] for c in cat_cols}).T
        unique_summary.columns = ['n_unique']
        display(unique_summary)
    else:
        print('No categorical cols detected.')
    # timestamp detection (automatic heuristics + candidate names)
    import re
    parsed = {}
    for c in df.columns:
        try:
            _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
            parsed[c] = True
        except Exception:
            parsed[c] = False
    timestamp_cols = [c for c, ok in parsed.items() if ok]
    likely_time = re.compile(r'(time|date|charttime|storetime|admittime|dischtime|intime|outtime|edregtime|edouttime|deathtime)$', re.IGNORECASE)
    candidate_ts = [c for c in df.columns if likely_time.search(c)]
    print('\nAuto-parsed timestamp columns:', timestamp_cols)
    print('Candidate timestamp column names by pattern:', candidate_ts)
    for c in list(dict.fromkeys(timestamp_cols + candidate_ts)):
        s = pd.to_datetime(df[c], errors='coerce')
        print(c, '| min:', s.min(), '| max:', s.max(), '| #na:', s.isna().sum())
    # Quick correlations (if numeric columns < 100)
    numcols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numcols) > 1 and len(numcols) <= 100:
        corr = df[numcols].corr()
        print('\nNumeric columns correlation matrix (top-left 6x6 preview):')
        display(corr.iloc[:6, :6])
    else:
        print('\nSkipping correlation matrix (too many or too few numeric columns).')
    # Save a small sample CSV for quick inspection (first 100 rows)
    out_sample = DATA_ROOT.parent / 'Work' / 'samples'
    out_sample.mkdir(parents=True, exist_ok=True)
    sample_file = out_sample / 'emar_sample_100.csv'
    df.head(100).to_csv(sample_file, index=False)
    print('Saved sample to:', sample_file)


File not found: Data\Raw_data\emar.csv


### Required Fields Analysis for `emar.csv`
Paste the output of the cell above and then fill the table below by hand or run automated reasoning prompts.

```
Required fields (suggested):
- Identifiers: subject_id, hadm_id, stay_id (if present)
- Timestamps: admittime, dischtime, intime, outtime, charttime, storetime, edregtime, edouttime
- Outcome markers: deathtime, hospital_expire_flag
- Clinical: for chartevents/labevents — itemid, value, valueuom, valuenum, charttime/storetime
- Metadata: admission_type, admission_location, discharge_location, insurance, ethnicity, language
```

**Why these fields?**
- Identifiers enable joins across datasets.
- Timestamps enable LOS and time-series alignment.
- Outcome markers are targets for modeling.
- Clinical numeric fields are the actual features used for vitals and labs.

**Why other fields are lower priority?**
- Billing or administrative fields often add noise and are not useful for physiological modeling.
- High-cardinality free-text fields require dedicated NLP preprocessing before use.


---

# Dataset: `microbiologyevents.csv`

**Goal:** Perform a clean EDA for `microbiologyevents.csv` following best-practice steps: setup & load, shape & dtypes, head/tail, missingness, descriptive stats, categorical unique counts, timestamp identification, and a short 'Required fields' analysis.


In [29]:
# ---- EDA for microbiologyevents.csv
file_path = DATA_ROOT / 'microbiologyevents.csv'
if not file_path.exists():
    print('File not found:', file_path)
else:
    print('Processing file:', file_path)
    df = pd.read_csv(file_path, low_memory=False)
    print('\nShape:', df.shape)
    display(df.head())
    display(df.tail())
    # dtypes
    dtypes = df.dtypes.astype(str).rename('dtype')
    display(dtypes.to_frame())
    # missingness
    missing_counts = df.isna().sum().sort_values(ascending=False)
    missing_pct = (df.isna().mean()*100).sort_values(ascending=False)
    missing_summary = pd.DataFrame({'missing_count': missing_counts, 'missing_pct': missing_pct.round(2)})
    display(missing_summary)
    # descriptive stats
    try:
        display(df.describe(include='number').T)
    except Exception as e:
        print('Numeric describe error:', e)
    try:
        display(df.describe(include='object').T)
    except Exception as e:
        print('Object describe error:', e)
    # categorical unique counts
    cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
    if cat_cols:
        unique_summary = pd.DataFrame({c: [df[c].nunique()] for c in cat_cols}).T
        unique_summary.columns = ['n_unique']
        display(unique_summary)
    else:
        print('No categorical cols detected.')
    # timestamp detection (automatic heuristics + candidate names)
    import re
    parsed = {}
    for c in df.columns:
        try:
            _ = pd.to_datetime(df[c], errors='raise', infer_datetime_format=True)
            parsed[c] = True
        except Exception:
            parsed[c] = False
    timestamp_cols = [c for c, ok in parsed.items() if ok]
    likely_time = re.compile(r'(time|date|charttime|storetime|admittime|dischtime|intime|outtime|edregtime|edouttime|deathtime)$', re.IGNORECASE)
    candidate_ts = [c for c in df.columns if likely_time.search(c)]
    print('\nAuto-parsed timestamp columns:', timestamp_cols)
    print('Candidate timestamp column names by pattern:', candidate_ts)
    for c in list(dict.fromkeys(timestamp_cols + candidate_ts)):
        s = pd.to_datetime(df[c], errors='coerce')
        print(c, '| min:', s.min(), '| max:', s.max(), '| #na:', s.isna().sum())
    # Quick correlations (if numeric columns < 100)
    numcols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numcols) > 1 and len(numcols) <= 100:
        corr = df[numcols].corr()
        print('\nNumeric columns correlation matrix (top-left 6x6 preview):')
        display(corr.iloc[:6, :6])
    else:
        print('\nSkipping correlation matrix (too many or too few numeric columns).')
    # Save a small sample CSV for quick inspection (first 100 rows)
    out_sample = DATA_ROOT.parent / 'Work' / 'samples'
    out_sample.mkdir(parents=True, exist_ok=True)
    sample_file = out_sample / 'microbiologyevents_sample_100.csv'
    df.head(100).to_csv(sample_file, index=False)
    print('Saved sample to:', sample_file)


File not found: Data\Raw_data\microbiologyevents.csv


### Required Fields Analysis for `microbiologyevents.csv`
Paste the output of the cell above and then fill the table below by hand or run automated reasoning prompts.

```
Required fields (suggested):
- Identifiers: subject_id, hadm_id, stay_id (if present)
- Timestamps: admittime, dischtime, intime, outtime, charttime, storetime, edregtime, edouttime
- Outcome markers: deathtime, hospital_expire_flag
- Clinical: for chartevents/labevents — itemid, value, valueuom, valuenum, charttime/storetime
- Metadata: admission_type, admission_location, discharge_location, insurance, ethnicity, language
```

**Why these fields?**
- Identifiers enable joins across datasets.
- Timestamps enable LOS and time-series alignment.
- Outcome markers are targets for modeling.
- Clinical numeric fields are the actual features used for vitals and labs.

**Why other fields are lower priority?**
- Billing or administrative fields often add noise and are not useful for physiological modeling.
- High-cardinality free-text fields require dedicated NLP preprocessing before use.


---

# Visualizations & Next Steps (placeholder cells)
The sections below contain code templates for common visualization and next-step processing. Run them after the dataset-specific cells above.


In [30]:
# Correlation Heatmap template (run on an in-memory df variable with numeric columns)
def plot_corr_heatmap(df, top_n=40):
    import matplotlib.pyplot as plt
    import seaborn as sns
    num = df.select_dtypes(include=[np.number])
    if num.shape[1] > top_n:
        # pick top_n by variance
        vars_sorted = num.var().sort_values(ascending=False).index[:top_n]
        num = num[vars_sorted]
    corr = num.corr()
    plt.figure(figsize=(10,8))
    sns.heatmap(corr, cmap='RdBu_r', center=0, annot=False)
    plt.title('Correlation heatmap (numeric columns)')
    plt.show()

print('plot_corr_heatmap is ready to use. Example: plot_corr_heatmap(df)')

plot_corr_heatmap is ready to use. Example: plot_corr_heatmap(df)


In [31]:
# Distribution plots for a list of columns (example for vitals)
def plot_distributions(df, cols, bins=50):
    import matplotlib.pyplot as plt
    import seaborn as sns
    for c in cols:
        if c in df.columns:
            plt.figure(figsize=(6,3))
            sns.histplot(df[c].dropna(), bins=bins, kde=True)
            plt.title(f'Distribution: {c} (n={df[c].notna().sum()})')
            plt.show()
        else:
            print('Column not in df:', c)

print('plot_distributions ready.')

plot_distributions ready.


In [16]:
# Time-series sample plotting (example for heart rate)
def plot_time_series(df, time_col, value_col, subject_col=None, hadm_col=None, n_points=1000):
    import matplotlib.pyplot as plt
    if subject_col and subject_col in df.columns:
        subj = df[subject_col].dropna().unique()[0]
        d = df[df[subject_col]==subj].copy()
    elif hadm_col and hadm_col in df.columns:
        hadm = df[hadm_col].dropna().unique()[0]
        d = df[df[hadm_col]==hadm].copy()
    else:
        d = df.copy()
    d[time_col] = pd.to_datetime(d[time_col], errors='coerce')
    d = d.sort_values(time_col).head(n_points)
    plt.figure(figsize=(12,3))
    plt.plot(d[time_col], d[value_col])
    plt.title('Time-series {} vs {}'.format(value_col, time_col))
    plt.xlabel(time_col)
    plt.ylabel(value_col)
    plt.show()

print('plot_time_series ready.')

plot_time_series ready.


## Tasks checklist (to run and fill)

- [ ] Deep dive into EDA for each dataset
- [ ] Map itemid → variable names using d_items.csv
- [ ] Identify important ICU vitals from chartevents.csv
- [ ] Filter ICU stays with enough data
- [ ] Resample to hourly bins and inspect gaps
- [ ] Propose imputation strategy (do not impute yet)
- [ ] Produce barplot: most frequent vital measurements
- [ ] Save final list of selected variables for modeling

---

Notebook generated programmatically. Save and run cells in sequence. Adjust DATA_ROOT path to point to your CSV folder if needed.


## Interpretation (Correlation Insights)
- Highly correlated: look for |r| ≥ 0.7; if present, consider dropping one feature or combining.
- Redundancy: IDs (e.g., `subject_id`, `hadm_id`) should be excluded from modeling; any strong correlation with outcomes is spurious.
- ICU modeling impact: time-related counts or engineered features may correlate with LOS or mortality; verify causality and leakage before using.

In [42]:
# Correlation Analysis Setup
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(context='notebook', style='whitegrid', font_scale=1.0)
plt.switch_backend('agg')  # use non-interactive backend
file_name = 'admissions.csv'  # change to target file
path = (DATA_ROOT / file_name)
df = pd.read_csv(path)
num_df = df.select_dtypes(include=['number']).copy()
print(f'Loaded {file_name} with numeric columns: {list(num_df.columns)}')
# Output directory for saved figures
OUT_DIR = Path('../Artifacts/plots')
OUT_DIR.mkdir(parents=True, exist_ok=True)

Loaded admissions.csv with numeric columns: ['subject_id', 'hadm_id', 'hospital_expire_flag']


In [43]:
# Correlation Matrix
corr = num_df.corr(numeric_only=True)
display(corr.round(3))

,subject_id,hadm_id,hospital_expire_flag
subject_id,1.000,0.018,0.017
hadm_id,0.018,1.000,-0.039
hospital_expire_flag,0.017,-0.039,1.000


In [44]:
# Annotated Heatmap (saved to file)
plt.figure(figsize=(max(6, 1.5*len(num_df.columns)), max(5, 1.2*len(num_df.columns))))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', vmin=-1, vmax=1, square=True)
plt.title(f'Correlation Heatmap: {file_name}')
plt.tight_layout()
out_heatmap = OUT_DIR / f"corr_heatmap_{file_name.replace('.csv','')}.png"
plt.savefig(out_heatmap, dpi=150)
print('Saved heatmap to:', out_heatmap)

Saved heatmap to: ..\Artifacts\plots\corr_heatmap_admissions.png


In [45]:
# Pairplot (feasible only for small number of numeric columns) - saved to file
cols = list(num_df.columns)
max_cols = 6
use_cols = cols[:max_cols]
if len(use_cols) >= 2:
    g = sns.pairplot(num_df[use_cols], corner=True, plot_kws={'s': 12, 'alpha': 0.6})
    g.fig.suptitle(f'Pairplot (first {max_cols} numeric cols): {file_name}', y=1.02)
    out_pair = OUT_DIR / f"pairplot_{file_name.replace('.csv','')}.png"
    g.fig.savefig(out_pair, dpi=150, bbox_inches='tight')
    print('Saved pairplot to:', out_pair)
else:
    print('Pairplot skipped: not enough numeric columns')

Saved pairplot to: ..\Artifacts\plots\pairplot_admissions.png


Vital Signs

In [46]:
# Vital Signs Extraction from chartevents + d_items
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
sns.set(context='notebook', style='whitegrid', font_scale=1.0)
OUT_DIR = Path('../Artifacts/plots')
OUT_DIR.mkdir(parents=True, exist_ok=True)
ce = pd.read_csv(DATA_ROOT / 'chartevents.csv', usecols=['subject_id','hadm_id','stay_id','charttime','itemid','valuenum','valueuom'])
di = pd.read_csv(DATA_ROOT.parent / 'Raw_data' / 'd_items.csv')
di['label_lower'] = di['label'].str.lower()
patterns = {
    'HR': ['heart rate','hr'],
    'SBP': ['systolic','sbp'],
    'DBP': ['diastolic','dbp'],
    'MAP': ['mean arterial pressure','map'],
    'SpO2': ['spo2','oxygen saturation','pulse ox'],
    'RR': ['respiratory rate','rr'],
    'Temp': ['temperature','temp']
}
vital_itemids = {}
for vital, pats in patterns.items():
    mask = di['label_lower'].apply(lambda x: any(p in x for p in pats))
    vital_itemids[vital] = set(di.loc[mask, 'itemid'].tolist())
print({k: len(v) for k,v in vital_itemids.items()})


{'HR': 67, 'SBP': 11, 'DBP': 11, 'MAP': 8, 'SpO2': 3, 'RR': 109, 'Temp': 57}


In [47]:
# Build long-form vital dataframe and plots
import numpy as np
def vital_series(vital):
    ids = vital_itemids.get(vital, set())
    if not ids:
        return pd.Series(dtype=float)
    s = ce.loc[ce['itemid'].isin(ids), 'valuenum']
    return pd.to_numeric(s, errors='coerce')
ranges = {
    'HR': (40, 180),
    'SBP': (70, 200),
    'DBP': (40, 120),
    'MAP': (55, 120),
    'SpO2': (70, 100),
    'RR': (8, 40),
    'Temp': (34, 41)
}
for vital in ['HR','SBP','DBP','MAP','SpO2','RR','Temp']:
    s = vital_series(vital).dropna()
    if s.empty:
        print(f'{vital}: no data found')
        continue
    skew = s.skew()
    low, high = ranges[vital]
    n = len(s)
    pct_low = (s < low).mean()*100
    pct_high = (s > high).mean()*100
    plt.figure(figsize=(8,5))
    sns.histplot(s, bins=50, kde=True, color='#2a9d8f')
    plt.title(f'{vital} distribution (n={n}, skew={skew:.2f})')
    plt.xlabel(vital)
    plt.ylabel('Count')
    out = OUT_DIR / f'vital_{vital}_dist.png'
    plt.tight_layout()
    plt.savefig(out, dpi=150)
    print(f'Saved {vital} plot to:', out)
    print(f'{vital} normal range approx: [{low}, {high}] | below: {pct_low:.1f}% | above: {pct_high:.1f}%')

Saved HR plot to: ..\Artifacts\plots\vital_HR_dist.png
HR normal range approx: [40, 180] | below: 4.4% | above: 0.0%
Saved SBP plot to: ..\Artifacts\plots\vital_SBP_dist.png
SBP normal range approx: [70, 200] | below: 2.8% | above: 0.1%
Saved DBP plot to: ..\Artifacts\plots\vital_DBP_dist.png
DBP normal range approx: [40, 120] | below: 4.7% | above: 0.3%
MAP: no data found
Saved SpO2 plot to: ..\Artifacts\plots\vital_SpO2_dist.png
SpO2 normal range approx: [70, 100] | below: 3.8% | above: 0.1%
Saved RR plot to: ..\Artifacts\plots\vital_RR_dist.png
RR normal range approx: [8, 40] | below: 13.8% | above: 1.1%
Saved Temp plot to: ..\Artifacts\plots\vital_Temp_dist.png
Temp normal range approx: [34, 41] | below: 13.1% | above: 58.1%
